In [1]:
# from transformers import BloomTokenizerFast, BloomForQuestionAnswering, BloomForCausalLM, TrainingArguments, Trainer
import logging
from dotenv import load_dotenv
from langchain.embeddings.cohere import CohereEmbeddings
import pinecone
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain import HuggingFaceHub
from langchain.prompts import PromptTemplate
from config.config import *

#utils
from src.utils import setup_logger, connect_index

# Memory
from langchain.memory import ConversationBufferMemory

load_dotenv()
logger = setup_logger(__name__, logging.DEBUG)

logger.info('Loading environment variables')


/Users/isaac/FundamentlPartners/abinvenv-sol/venv/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
2023-07-11 18:59:18,482 - src.utils - DEBUG - Setting up logger


In [2]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [3]:
embeddings = CohereEmbeddings(model=COHERE_MODEL_NAME, cohere_api_key=COHERE_API_KEY)

In [12]:
index = connect_index(PINECONE_INDEX_NAME)
vectorstore = Pinecone(index, embeddings.embed_query, "text")

2023-07-11 19:02:32,882 - src.utils - INFO - Connected to Pinecone index stab-test


In [20]:
query = "How can i predict rainfall?"
docs = vectorstore.max_marginal_relevance_search(query=query, top_k=5)
docs

[Document(page_content='5 CENTRE FOR HUMANITARIAN DATA• Was the rainfall last month above or below normal?  And by how much?  Observational \nrainfall data goes back decades and can therefore be used to determine whether rainfall for \na month or a season is above, at or below the historical average. \n• Were there more rainy days this month than last month? Observational rainfall data is \navailable for each day and can be used to determine if it rained or not on a given day and to \ncount the number of dry or wet days during a specific period of time. \n• How many dry spells were there during the past rainy season? How long was the \nlongest dry spell and the heaviest rain streak this past rainy season?  Observational \nrainfall data can be used to detect the occurrence, location and duration of dry spells and \nprolonged periods of heavy rain.  \n• How often in the past decade were more than x millimeters of rainfall measured?  \nObservational rainfall data provides daily rainfall t

In [10]:
template = """You are a chatbot having a conversation with a human.

Given the following extracted parts of a long document and a question, create a final answer.

{context}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")
chain = load_qa_chain(
    OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY), chain_type="stuff", memory=memory, prompt=prompt
)

In [22]:
query
chain({"input_documents": docs, "human_input": query}, return_only_outputs=False)

{'input_documents': [Document(page_content='5 CENTRE FOR HUMANITARIAN DATA• Was the rainfall last month above or below normal?  And by how much?  Observational \nrainfall data goes back decades and can therefore be used to determine whether rainfall for \na month or a season is above, at or below the historical average. \n• Were there more rainy days this month than last month? Observational rainfall data is \navailable for each day and can be used to determine if it rained or not on a given day and to \ncount the number of dry or wet days during a specific period of time. \n• How many dry spells were there during the past rainy season? How long was the \nlongest dry spell and the heaviest rain streak this past rainy season?  Observational \nrainfall data can be used to detect the occurrence, location and duration of dry spells and \nprolonged periods of heavy rain.  \n• How often in the past decade were more than x millimeters of rainfall measured?  \nObservational rainfall data provi